In [2]:
%pip install thefuzz

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 1.6/1.6 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from thefuzz import fuzz  # For Thai text fuzzy matching
from thefuzz import process

def fuzzy_merge_locations():
    # Load data
    print("Loading data...")
    fossil_list = pd.read_csv('data/fossil_list.csv')
    jurassic_data = pd.read_csv('data/jurrasic raw.csv')
    
    # Remove 'attributes_' prefix
    jurassic_data.columns = [col.replace('attributes_', '') for col in jurassic_data.columns]
    
    print("\nInitial data:")
    print(f"Fossil list: {len(fossil_list)} rows")
    print(f"Jurassic data: {len(jurassic_data)} rows")

    # Function to find best match using fuzzy matching
    def find_best_match(row, master_locations, threshold=80):
        best_match = process.extractOne(
            f"{row['PROVINCE']}-{row['DISTRICT']}-{row['TAMBOM']}",
            master_locations,
            scorer=fuzz.token_sort_ratio
        )
        if best_match and best_match[1] >= threshold:
            return best_match[0]
        return None

    # Create location strings for matching
    fossil_locations = fossil_list.apply(
        lambda x: f"{x['PROVINCE']}-{x['DISTRICT']}-{x['TAMBOM']}", axis=1
    ).unique()
    
    jurassic_locations = jurassic_data.apply(
        lambda x: f"{x['PROVINCE']}-{x['DISTRICT']}-{x['TAMBON']}", axis=1
    ).unique()

    print("\nUnique locations before fuzzy matching:")
    print(f"Fossil locations: {len(fossil_locations)}")
    print(f"Jurassic locations: {len(jurassic_locations)}")

    # Create mapping dictionary using fuzzy matching
    location_mapping = {}
    for fossil_loc in fossil_locations:
        match = process.extractOne(
            fossil_loc,
            jurassic_locations,
            scorer=fuzz.token_sort_ratio
        )
        if match and match[1] >= 80:  # 80% similarity threshold
            location_mapping[fossil_loc] = match[0]

    print("\nFuzzy matching results:")
    print(f"Matched locations: {len(location_mapping)}")

    # Apply mapping to fossil data
    fossil_list['location_key'] = fossil_list.apply(
        lambda x: f"{x['PROVINCE']}-{x['DISTRICT']}-{x['TAMBOM']}", axis=1
    ).map(location_mapping)

    jurassic_data['location_key'] = jurassic_data.apply(
        lambda x: f"{x['PROVINCE']}-{x['DISTRICT']}-{x['TAMBON']}", axis=1
    )

    # Perform merge using matched locations
    fuzzy_merged = pd.merge(
        fossil_list,
        jurassic_data,
        on='location_key',
        how='left',
        suffixes=('_fossil', '_site')
    )

    # Analyze results
    print("\nMerge results:")
    print(f"Total rows after fuzzy merge: {len(fuzzy_merged)}")
    
    matched_rows = fuzzy_merged[fuzzy_merged['SITE_ID'].notna()]
    unmatched_rows = fuzzy_merged[fuzzy_merged['SITE_ID'].isna()]
    
    print(f"Matched rows: {len(matched_rows)}")
    print(f"Unmatched rows: {len(unmatched_rows)}")

    # Sample of fuzzy matches
    print("\nSample of successful fuzzy matches:")
    sample_matches = matched_rows[['PROVINCE_fossil', 'DISTRICT_fossil', 'TAMBOM', 
                                 'PROVINCE_site', 'DISTRICT_site', 'TAMBON']].head()
    print(sample_matches)

    # Sample of unmatched locations
    print("\nSample of unmatched locations:")
    unmatched_locations = unmatched_rows[['PROVINCE_fossil', 'DISTRICT_fossil', 'TAMBOM']].head()
    print(unmatched_locations)

    # Export results
    fuzzy_merged.to_csv('fuzzy_merged_data.csv', index=False)
    matched_rows.to_csv('fuzzy_matched.csv', index=False)
    unmatched_rows.to_csv('fuzzy_unmatched.csv', index=False)

    return {
        'merged': fuzzy_merged,
        'matched': matched_rows,
        'unmatched': unmatched_rows,
        'mapping': location_mapping
    }

# Additional function to analyze mapping quality
def analyze_mapping_quality(mapping):
    print("\nAnalyzing mapping quality:")
    
    similarity_scores = []
    for source, target in mapping.items():
        score = fuzz.token_sort_ratio(source, target)
        similarity_scores.append({
            'source': source,
            'target': target,
            'similarity': score
        })
    
    df_scores = pd.DataFrame(similarity_scores)
    
    print("\nSimilarity score distribution:")
    print(df_scores['similarity'].describe())
    
    print("\nSample of high similarity matches (>95%):")
    print(df_scores[df_scores['similarity'] > 95].head())
    
    print("\nSample of lower similarity matches (80-85%):")
    print(df_scores[(df_scores['similarity'] >= 80) & 
                   (df_scores['similarity'] < 85)].head())

# Run the fuzzy merge
results = fuzzy_merge_locations()

# Analyze the mapping quality
analyze_mapping_quality(results['mapping'])

Loading data...

Initial data:
Fossil list: 7198 rows
Jurassic data: 520 rows

Unique locations before fuzzy matching:
Fossil locations: 455
Jurassic locations: 279

Fuzzy matching results:
Matched locations: 121

Merge results:
Total rows after fuzzy merge: 15409
Matched rows: 12016
Unmatched rows: 3393

Sample of successful fuzzy matches:
  PROVINCE_fossil DISTRICT_fossil       TAMBOM PROVINCE_site DISTRICT_site  \
4           พังงา         เกาะยาว  เกาะยาวใหญ่         พังงา       เกาะยาว   
5           พังงา         เกาะยาว  เกาะยาวใหญ่         พังงา       เกาะยาว   
6           พังงา         เกาะยาว  เกาะยาวใหญ่         พังงา       เกาะยาว   
7           พังงา         เกาะยาว  เกาะยาวใหญ่         พังงา       เกาะยาว   
8           พังงา         เกาะยาว  เกาะยาวใหญ่         พังงา       เกาะยาว   

        TAMBON  
4  เกาะยาวน้อย  
5  เกาะยาวน้อย  
6  เกาะยาวน้อย  
7  เกาะยาวน้อย  
8  เกาะยาวน้อย  

Sample of unmatched locations:
   PROVINCE_fossil DISTRICT_fossil    TAMBOM
0        

In [10]:
import pandas as pd
df = pd.read_csv('fuzzy_matched.csv')
pd.set_option('display.max_columns', None)
df

,FOSSIL_ID,SCI_NAME,COM_NAME,F_GROUP,F_TYPE,F_PART,DIS_NAME,PROVINCE_fossil,DISTRICT_fossil,TAMBOM,REGIS_CODE,location_key,SITE_ID,SITE_CODE,NAME_TH,NAME_ENG,IMPORTA_TH,IMPORTA_EN,LOCALITY,TAMBON,DISTRICT_site,PROVINCE_site,UTM_E,UTM_N,ZONE,MAPSHEET,SHEET_NAME,FEATURE_TH,FEATURE_EN,GEO_DES_TH,GEO_DES_EN,GEO_GROUP,FORMATION,PERIODFROM,PERIODTO,FOS_DES_TH,FOS_GR_1,FOS_GR_2,FOS_GR_3,FOS_GR_4,FOS_GR_5,DIS_BY_ENG,DIS_BY_TH,DIS_DAY,DIS_MONTH,DIS_YEAR,OWNER_TH,OWNER_ENG,POTENTIAL1,POTENTIAL2,POTENTIAL3,POTENTIAL4,POTENTIAL5,POTENTIAL6,POTENTIAL7,STATUS,CREATEBY,CREATEDATE,UPDATEBY,UPDATEDATE,GlobalID,OBJECTID,created_user,created_date,last_edited_user,last_edited_date,geometry_x,geometry_y
0,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,THF 2561 2 00199,พังงา-เกาะยาว-เกาะยาวน้อย,369.0,182002.0,แบรคิโอพอดควนจุก เกาะยาวน้อย,NaN,แหล่งอ้างอิงทางวิชาการ,Reference site,เกาะยาวน้อย,เกาะยาวน้อย,เกาะยาว,พังงา,455947.0,900155.0,47N,4725 III,กิ่งอำเภอเกาะยาว,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,แบรคิโอพอด,N,Invertebrate,N,N,N,NaN,NaN,NaN,NaN,NaN,กรมป่าไม้,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1751.0,1.696261e+12,NaN,NaN,{02C27880-6BA4-4CFD-97C6-BE452974953E},11247.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,455947.0,900155.0
1,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,THF 2561 2 00199,พังงา-เกาะยาว-เกาะยาวน้อย,66.0,182009.0,แบรคิโอพอดโล๊ะมุน,NaN,เตรียมการประกาศเป็นเขตสำรวจ ศึกษาวิจัย,Preparing for study site,แหลมโล๊ะหมุน,เกาะยาวน้อย,เกาะยาว,พังงา,458554.0,897837.0,47N,4725 III,กิ่งอำเภอเกาะยาว,ไหล่ทวีป,NaN,หินทรายขนาด fine - medium grained ชั้นหินหนาถึ...,NaN,แก่งกระจาน,เขาพระ/เกาะยาวน้อย,เพอร์เมียน,NaN,แบรคิโอพอด โดดเด่น พบรูปแบบที่แตกต่างกันอย่างน...,N,Invertebrate,N,N,N,NaN,NaN,NaN,NaN,NaN,กรมป่าไม้,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1751.0,1.696261e+12,NaN,NaN,{307AEFC8-3A20-4FB0-9454-2C4D84F80F61},11319.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,458554.0,897837.0
2,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,THF 2561 2 00199,พังงา-เกาะยาว-เกาะยาวน้อย,67.0,182010.0,แบรคิโอพอดบ่อน้ำศักดิ์สิทธิ์,NaN,เตรียมการประกาศเป็นเขตสำรวจ ศึกษาวิจัย,Preparing for study site,บ่อน้ำศักดิ์สิทธิ์,เกาะยาวน้อย,เกาะยาว,พังงา,456229.0,901537.0,47N,4725 III,กิ่งอำเภอเกาะยาว,ไหล่ทวีป,NaN,หินทรายขนาด fine - very fine grained สีผุ สีน้...,NaN,แก่งกระจาน,เขาพระ/เกาะยาวน้อย,เพอร์เมียน,NaN,แบรคิโอพอด โดดเด่น พบรูปแบบที่แตกต่างกันอย่างน...,N,Invertebrate,N,N,N,NaN,NaN,NaN,NaN,NaN,กรมป่าไม้,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1751.0,1.696261e+12,NaN,NaN,{8533F56F-4FA2-40D2-8376-7A585F5CB7CE},11320.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,456229.0,901537.0
3,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,THF 2561 2 00199,พังงา-เกาะยาว-เกาะยาวน้อย,498.0,182012.0,ปะการังบ้านน้ำจืด,NaN,แหล่งอ้างอิงทางวิชาการ,Reference site,NaN,เกาะยาวน้อย,เกาะยาว,พังงา,458300.0,901301.0,47N,4725 III,กิ่งอำเภอเกาะยาว,ไหล่ทวีป,NaN,หินปูนสีเทาอ่อน บางส่วนมีการตกผลิกใหม่,NaN,ราชบุรี,อุ้มลูก,เพอร์เมียน,NaN,ปะการัง และไครนอยด์,N,Invertebrate,N,N,N,NaN,NaN,NaN,NaN,NaN,กรมป่าไม้,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1751.0,1.696261e+12,NaN,NaN,{75F68926-73C3-4001-AD69-DDE48D55D316},11539.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,458300.0,901301.0
4,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,THF 2561 2 00199,พังงา-เกาะยาว-เกาะยาวน้อ